In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [42]:
def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS

def crossValidationPoKolei(clf, data, dataClass):
    err = 0
    for i in range(data.shape[0]):
        pojedynczyErr = 0
        ilosc = 0
        for j in range(data.shape[1]):
            X_train = data[i][j][0]
            y_train = dataClass[i][j][0]
            X_test = data[i][j][1]
            y_test = dataClass[i][j][1] 
            pojIlosc = len(y_test)
            ilosc += pojIlosc
            pojedynczyErr += pojIlosc * licz(X_train, X_test, y_train, y_test, clf)

        err += pojedynczyErr / ilosc
    return float(err) / data.shape[0]

def przelicz(xTrain, xTest, yTrain, yTest, kmeans, clf, kmeansData):
    trainLabels = kmeansData[0]
    testLabels = kmeansData[1]
    k = kmeans.n_clusters
    wynik = 0
    for i in range(k):
        iloscTestowych = np.sum(testLabels == i)
        if (np.sum(trainLabels == i) > 1 and iloscTestowych > 1):
            wynik += iloscTestowych * licz(xTrain[trainLabels==i], xTest[testLabels==i], yTrain[trainLabels==i], yTest[testLabels==i], clf)
    return wynik / yTest.shape[0]

def licz(xTrain, xTest, yTrain, yTest, clf):
    if(len(yTest) == 0):
        return 0
    if (len(yTrain) == 0):
        wynik = np.sum(yTest == 1) / yTest.shape[0]
        return wynik
    if (np.unique(yTrain).shape[0] == 1):
        wynik = np.sum(yTest == yTrain[0]) / yTest.shape[0]
        return wynik
    
    clf = clf.fit(xTrain, yTrain)
    yTrain = yTrain.reshape((yTrain.shape[0], 1))
    wynikDecFun = clf.decision_function(xTrain).reshape(yTrain.shape)
    najDecFun = najlepszeDecFun(yTrain, wynikDecFun)  
    return testujDecisionFunction(xTest, yTest, najDecFun, clf)

def generujPodzialy(tab, p = 0.75):
    podzial = []
    n = len(tab)
    k = int(10*n/100)
    dlugosc = int(4/10*n)
    dlugoscUczaca = int(4/10*p*n)
    for i in range(6):
        uczace = tab[np.arange(i*k, min(i*k + dlugoscUczaca, n))]
        testowe = tab[np.arange(min(i*k + dlugoscUczaca, n), min((i*k + dlugosc), n))]
        podzial.append([uczace, testowe])
    return podzial

def stworzGrupowanie(kmeans, tabDlaKmeans, tabDlaSvm, zmiana):
    podzialSvm = generujPodzialy(tabDlaSvm)
    podzialKmeans = generujPodzialy(tabDlaKmeans)
    podzialZmiana = generujPodzialy(zmiana)
    
    wynik = []
    wynikZmiana = []
    for j in range(len(podzialKmeans)):
        xKmeansTrain = podzialKmeans[j][0]
        xKmeansTest = podzialKmeans[j][1]
        zmianaTrain = podzialZmiana[j][0]
        zmianaTest = podzialZmiana[j][1]
        svmTrain = podzialSvm[j][0]
        svmTest = podzialSvm[j][1]
        kmeans.fit(xKmeansTrain)
        trainLabels = kmeans.labels_
        testLabels = kmeans.predict(xKmeansTest)
        pojedynczyWynik = []
        pojedynczyWynikZmiana = []
        for i in range(kmeans.n_clusters):
            pojedynczyWynik.append([svmTrain[i == trainLabels], svmTest[i == testLabels]])
            pojedynczyWynikZmiana.append([zmianaTrain[i == trainLabels], zmianaTest[i == testLabels]])
        wynik.append(pojedynczyWynik)
        wynikZmiana.append(pojedynczyWynikZmiana)
    return np.array(wynik), np.array(wynikZmiana)

def rodzaj(x):
    return {
        0 : 'Czyste',
        1 : 'S',
        2 : 'N',
        3 : 'S->N',
        4 : 'N->S'
    }[x]

In [81]:
def test(mozliwosc, svm):
    wynik1 = crossValidationPoKolei(svm, mozliwosc[0][0], mozliwosc[0][1])
    wynik2 = crossValidationPoKolei(svm, mozliwosc[1][0], mozliwosc[1][1])
    wynik3 = crossValidationPoKolei(svm, mozliwosc[2][0], mozliwosc[2][1])
    wynik4= crossValidationPoKolei(svm, mozliwosc[3][0], mozliwosc[3][1])
    wynik5 = crossValidationPoKolei(svm, mozliwosc[4][0], mozliwosc[4][1])
    crossValidationPoKolei(svm, gPodzial, gZmiana)
    return wynik1, wynik2, wynik3, wynik4, wynik5

def wylicz(tabSvm, tabKmeans, zmiana, tabParametrow, k, iloscTestow):
    listaTabKmeans = standaryzowanie(tabKmeans)
    listaTabSvm = standaryzowanie(tabSvm)
    
    tablicaTestow = []
    for a in range(iloscTestow):
        tablicaMozliwosci = []
        for i in range(5):
            mozliwosc = []
            for j in range(5):
                kmeans = KMeans(init='k-means++', n_clusters=k, n_init=15, random_state=a*12345)
                mozliwosc.append(stworzGrupowanie(kmeans, listaTabKmeans[i], listaTabSvm[j], zmiana))
            tablicaMozliwosci.append(mozliwosc)
        tablicaTestow.append(tablicaMozliwosci)
    
    tabWynikow = np.zeros((len(tabParametrow)*5,5), dtype=np.float32)

    for tablicaMozliwosci in tablicaTestow:
        i = 0
        for parametry in tabParametrow:
            gamma = parametry[0]
            C = parametry[1]
            j = 0
            for mozliwosc in tablicaMozliwosci:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = 'rbf', C = C, random_state=1)
                wynikTestu = test(mozliwosc, svm)
                tabWynikow[i+j] += wynikTestu
                j += 1
            i  += 5
    return tabWynikow / iloscTestow

def wypisz(tabWynikow):
    print('Czyste     S      N     S->N    N->S')
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        for j in range(5):
            wynik1, wynik2, wynik3, wynik4, wynik5 = tabWynikow[i+j]
            odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
            if (wynik1 >= 0.544 or wynik2 >= 0.544 or wynik3 >= 0.544 or wynik4 > 0.544 or wynik5 >= 0.544):
                odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
            print(odpowiedz, "- dla: g:", gamma, "C:", C, "daneKmeans:", rodzaj(j)) 
        print()
        i += 5

In [67]:
gmm_10_M_fPierw = np.load("gmm_10_M_fPierw.npy")
zmiana = np.load("zmiana.npy")
imbalance = np.load("imbalance.npy")

gammas = [1e-6, 1e-3, 1, ]
Cs = [1e-6, 1e-3, 1, 1e3]
tabParametrow = np.zeros((len(gammas)*len(Cs),2), dtype=np.float64)
i = 0
for gamma in gammas:
    for C in Cs:
        tabParametrow[i][0] = gamma
        tabParametrow[i][1] = C
        i += 1

In [94]:
t = time.time()
means2_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 2, 10)
print(time.time() - t)
np.save("means2_imbalance_gmm_10_test3.npy", means2_imbalance_gmm_10Wynik)
wypisz(means2_imbalance_gmm_10Wynik)

1940.6413860321045
Czyste     S      N     S->N    N->S
"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
"0.5443  0.5443  0.5421  0.5418  0.5421" - dla: g: 1e-06 C: 1e-06 daneKmeans: S
"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5428  0.5428  0.5356  0.5352  0.5356 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
"0.5443  0.5443  0.5421  0.5418  0.5421" - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
"0.5443  0.5443  0.5421  0.5418  0.5421" - dla: g: 1e-06 C: 0.001 daneKmeans: S
"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5428  0.5428  0.5356  0.5352  0.5356 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
"0.5443  0.5443  0.5421  0.5418  0.5421" - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
"0.5443  0.5443  0.5421  0.5418  0.5421" 

In [95]:
t = time.time()
means3_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 3, 10)
print(time.time() - t)
np.save("means3_imbalance_gmm_10_test3.npy", means3_imbalance_gmm_10Wynik)
wypisz(means3_imbalance_gmm_10Wynik)

1350.7587962150574
Czyste     S      N     S->N    N->S
0.5352  0.5352  0.5418  0.5428  0.5418 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5363  0.5363  0.5356  0.5345  0.5356 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5350  0.5350  0.5411  0.5422  0.5411 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5301  0.5301  0.5261  0.5261  0.5261 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5359  0.5359  0.5356  0.5345  0.5356 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5352  0.5352  0.5418  0.5428  0.5418 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5363  0.5363  0.5356  0.5345  0.5356 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5350  0.5350  0.5411  0.5422  0.5411 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5301  0.5301  0.5261  0.5261  0.5261 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5359  0.5359  0.5356  0.5345  0.5356 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5352  0.5352  0.5418  0.5428  0.5418 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5363  0.5363  0.5356  0.5345  0.5356 - dla: g: 1e-06 C: 1

In [96]:
t = time.time()
means4_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 4, 10)
print(time.time() - t)
np.save("means4_imbalance_gmm_10_test3.npy", means4_imbalance_gmm_10Wynik)
wypisz(means4_imbalance_gmm_10Wynik)

1247.462319135666
Czyste     S      N     S->N    N->S
"0.5446  0.5446  0.5374  0.5385  0.5374" - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5336  0.5336  0.5236  0.5295  0.5236 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5435  0.5435  0.5364  0.5375  0.5364 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5199  0.5199  0.5162  0.5214  0.5162 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5304  0.5304  0.5211  0.5280  0.5211 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

"0.5446  0.5446  0.5374  0.5385  0.5374" - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5336  0.5336  0.5236  0.5295  0.5236 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5435  0.5435  0.5364  0.5375  0.5364 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5199  0.5199  0.5162  0.5214  0.5162 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5304  0.5304  0.5211  0.5280  0.5211 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

"0.5446  0.5446  0.5374  0.5385  0.5374" - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5336  0.5336  0.5236  0.5295  0.5236 - dla: g: 1e-06

In [97]:
t = time.time()
means5_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 5 , 10)
print(time.time() - t)
np.save("means5_imbalance_gmm_10_test3.npy", means5_imbalance_gmm_10Wynik)
wypisz(means5_imbalance_gmm_10Wynik)

1123.5635962486267
Czyste     S      N     S->N    N->S
0.5273  0.5273  0.5364  0.5398  0.5364 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5216  0.5216  0.5279  0.5290  0.5279 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5262  0.5262  0.5376  0.5401  0.5376 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5172  0.5172  0.5221  0.5276  0.5221 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5244  0.5243  0.5243  0.5310  0.5243 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5273  0.5273  0.5364  0.5398  0.5364 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5216  0.5216  0.5279  0.5290  0.5279 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5262  0.5262  0.5376  0.5401  0.5376 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5172  0.5172  0.5221  0.5276  0.5221 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5243  0.5243  0.5243  0.5310  0.5243 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5273  0.5273  0.5364  0.5398  0.5364 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5216  0.5216  0.5279  0.5290  0.5279 - dla: g: 1e-06 C: 1

In [98]:
t = time.time()
means6_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 6, 10)
print(time.time() - t)
np.save("means6_imbalance_gmm_10_test3.npy", means6_imbalance_gmm_10Wynik)
wypisz(means6_imbalance_gmm_10Wynik)

1060.480435371399
Czyste     S      N     S->N    N->S
0.5212  0.5212  0.5354  0.5340  0.5354 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5147  0.5147  0.5142  0.5200  0.5142 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5185  0.5185  0.5381  0.5314  0.5381 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5263  0.5263  0.5236  0.5228  0.5236 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5208  0.5208  0.5160  0.5184  0.5160 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5212  0.5212  0.5354  0.5340  0.5354 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5147  0.5147  0.5142  0.5200  0.5142 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5185  0.5185  0.5381  0.5314  0.5381 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5263  0.5263  0.5236  0.5228  0.5236 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5208  0.5208  0.5160  0.5184  0.5160 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5212  0.5212  0.5354  0.5340  0.5354 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5147  0.5147  0.5142  0.5200  0.5142 - dla: g: 1e-06 C: 1.

In [99]:
t = time.time()
means7_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 7, 10)
print(time.time() - t)
np.save("means7_imbalance_gmm_10_test3.npy", means7_imbalance_gmm_10Wynik)
wypisz(means7_imbalance_gmm_10Wynik)

1010.4583122730255
Czyste     S      N     S->N    N->S
0.5060  0.5060  0.5279  0.5283  0.5279 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5146  0.5146  0.5203  0.5187  0.5203 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5157  0.5157  0.5310  0.5295  0.5310 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5251  0.5251  0.5250  0.5192  0.5250 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5088  0.5088  0.5197  0.5185  0.5197 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5060  0.5060  0.5279  0.5283  0.5279 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5146  0.5146  0.5203  0.5187  0.5203 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5157  0.5157  0.5310  0.5295  0.5309 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5251  0.5251  0.5250  0.5192  0.5250 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5088  0.5088  0.5197  0.5185  0.5197 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5060  0.5060  0.5279  0.5283  0.5279 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5146  0.5146  0.5203  0.5187  0.5203 - dla: g: 1e-06 C: 1

In [100]:
t = time.time()
means8_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 8, 10)
print(time.time() - t)
np.save("means8_imbalance_gmm_10_test3.npy", means8_imbalance_gmm_10Wynik)
wypisz(means8_imbalance_gmm_10Wynik)

969.4120457172394
Czyste     S      N     S->N    N->S
0.5064  0.5064  0.5315  0.5322  0.5315 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5235  0.5235  0.5231  0.5216  0.5231 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5060  0.5060  0.5310  0.5308  0.5310 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5187  0.5187  0.5246  0.5182  0.5246 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5194  0.5194  0.5229  0.5208  0.5229 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5064  0.5064  0.5315  0.5322  0.5314 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5235  0.5235  0.5231  0.5216  0.5231 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5060  0.5060  0.5310  0.5308  0.5307 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5187  0.5187  0.5246  0.5182  0.5246 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5194  0.5194  0.5229  0.5208  0.5229 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5064  0.5064  0.5314  0.5322  0.5314 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5235  0.5235  0.5231  0.5216  0.5231 - dla: g: 1e-06 C: 1.

In [101]:
t = time.time()
means9_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 9, 10)
print(time.time() - t)
np.save("means9_imbalance_gmm_10_test3.npy", means9_imbalance_gmm_10Wynik)
wypisz(means9_imbalance_gmm_10Wynik)

937.5311892032623
Czyste     S      N     S->N    N->S
0.5093  0.5093  0.5312  0.5324  0.5312 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5146  0.5146  0.5156  0.5130  0.5156 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5033  0.5033  0.5290  0.5313  0.5290 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5134  0.5134  0.5142  0.5129  0.5142 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5108  0.5108  0.5210  0.5170  0.5210 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5093  0.5093  0.5312  0.5324  0.5309 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5146  0.5146  0.5156  0.5130  0.5156 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5033  0.5033  0.5290  0.5313  0.5285 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5134  0.5134  0.5142  0.5129  0.5142 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5108  0.5108  0.5210  0.5170  0.5210 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5093  0.5093  0.5309  0.5326  0.5309 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5146  0.5146  0.5156  0.5130  0.5156 - dla: g: 1e-06 C: 1.

In [102]:
t = time.time()
means10_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 10, 10)
print(time.time() - t)
np.save("means10_imbalance_gmm_10_test3.npy", means10_imbalance_gmm_10Wynik)
wypisz(means10_imbalance_gmm_10Wynik)

914.4281666278839
Czyste     S      N     S->N    N->S
0.5058  0.5058  0.5303  0.5319  0.5303 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5082  0.5081  0.5199  0.5164  0.5199 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5094  0.5094  0.5295  0.5308  0.5295 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5184  0.5184  0.5138  0.5134  0.5138 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5069  0.5061  0.5215  0.5175  0.5215 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5058  0.5058  0.5303  0.5319  0.5302 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5081  0.5081  0.5199  0.5164  0.5199 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5094  0.5094  0.5295  0.5308  0.5294 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5184  0.5184  0.5138  0.5134  0.5138 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5061  0.5061  0.5215  0.5175  0.5215 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5058  0.5058  0.5302  0.5319  0.5302 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5081  0.5081  0.5199  0.5164  0.5199 - dla: g: 1e-06 C: 1.

In [103]:
t = time.time()
means11_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 11, 10)
print(time.time() - t)
np.save("means11_imbalance_gmm_10_test3.npy", means11_imbalance_gmm_10Wynik)
wypisz(means11_imbalance_gmm_10Wynik)

908.0003616809845
Czyste     S      N     S->N    N->S
0.5105  0.5105  0.5278  0.5302  0.5278 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5009  0.5009  0.5211  0.5165  0.5211 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5135  0.5135  0.5260  0.5281  0.5260 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5197  0.5197  0.5131  0.5134  0.5131 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5040  0.5027  0.5204  0.5146  0.5204 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5105  0.5105  0.5278  0.5302  0.5276 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5009  0.5009  0.5211  0.5165  0.5211 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5135  0.5135  0.5260  0.5281  0.5257 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5197  0.5197  0.5131  0.5134  0.5131 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5027  0.5027  0.5204  0.5146  0.5204 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5105  0.5105  0.5276  0.5302  0.5276 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5009  0.5009  0.5211  0.5165  0.5211 - dla: g: 1e-06 C: 1.

In [104]:
t = time.time()
means12_imbalance_gmm_10Wynik = wylicz(imbalance, gmm_10_M_fPierw, zmiana, tabParametrow, 12, 10)
print(time.time() - t)
np.save("means12_imbalance_gmm_10_test3.npy", means12_imbalance_gmm_10Wynik)
wypisz(means12_imbalance_gmm_10Wynik)

907.7450799942017
Czyste     S      N     S->N    N->S
0.5134  0.5135  0.5282  0.5231  0.5282 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5015  0.5015  0.5207  0.5147  0.5207 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5124  0.5124  0.5272  0.5257  0.5272 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5180  0.5180  0.5154  0.5149  0.5154 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5038  0.5027  0.5236  0.5175  0.5236 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5135  0.5135  0.5282  0.5231  0.5281 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5015  0.5015  0.5207  0.5147  0.5207 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5124  0.5124  0.5272  0.5257  0.5272 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5180  0.5180  0.5154  0.5149  0.5154 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5027  0.5027  0.5236  0.5175  0.5236 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5135  0.5135  0.5281  0.5231  0.5281 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5015  0.5015  0.5207  0.5147  0.5207 - dla: g: 1e-06 C: 1.